# import libraries

In [2]:
! pip install beautifulsoup4


In [3]:
! pip install lxml
! pip install requests

In [4]:
!pip install html5lib
import html.parser

In [5]:
from bs4 import BeautifulSoup
import requests


In [6]:
! pip install urllib3
import urllib3.request

In [7]:
import pandas as pd
import numpy as np

# scrape table from wikipedia

In [8]:
url1 = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup1 = BeautifulSoup(response, 'lxml')
table1 = soup1.find('div', id = 'mw-content-text')
rows = table1.find_all('tr')
columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]

In [9]:
df = pd.DataFrame(columns=columns)
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    if len(tds) == 3:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n','')]
    else:
        pass
    df = df.append(pd.Series(values, index = columns), ignore_index=True)
    df.to_csv('kanchan2.csv')

In [8]:
df = pd.read_csv('kanchan.csv')
df.shape

(288, 3)

In [9]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# drop rows from the "Borough" which has values "not assigned"

In [14]:
#drop the row if borrow == "not assigned"
df['Borough'].replace('Not assigned',np.nan,inplace = True)
df.dropna(subset=['Borough'], axis = 0, inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


# group the duplicate rows

In [41]:
#group neighbourhoods in the same postcode
df_grouped = df.groupby(["Postcode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.tail(20)

,Postcode,Borough,Neighbourhood
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Runnymede, Swansea"
85,M7A,Queen's Park,Not assigned
86,M7R,Mississauga,Canada Post Gateway Processing Centre
87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
89,M8W,Etobicoke,"Alderwood, Long Branch"
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
92,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


# replace neighbourhood "Not assigned" by the values from the borough

In [46]:
# if Neighbourhood="Not assigned", the value is same as Borough
for index, row in df_grouped.iterrows():
    if row["Neighbourhood"] == "Not assigned":
        row["Neighbourhood"] = row["Borough"]
        
df_grouped.tail(20)

,Postcode,Borough,Neighbourhood
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Runnymede, Swansea"
85,M7A,Queen's Park,Queen's Park
86,M7R,Mississauga,Canada Post Gateway Processing Centre
87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
89,M8W,Etobicoke,"Alderwood, Long Branch"
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
92,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


# print the shape of the cleaned data

In [47]:
df_grouped.shape

(103, 3)